In [1]:
import requests, csv, tkinter as tk
from tkinter import ttk, messagebox, filedialog
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image, ImageTk
import io

# ------------------ Constants ------------------
SOLVENTS = {
    "Water": {"eps": 80.1, "type": "polar_protic"},
    "Methanol": {"eps": 32.6, "type": "polar_protic"},
    "Ethanol": {"eps": 24.6, "type": "polar_protic"},
    "Acetone": {"eps": 20.7, "type": "polar_aprotic"},
    "DMSO": {"eps": 47.2, "type": "polar_aprotic"},
    "Acetonitrile": {"eps": 37.5, "type": "polar_aprotic"},
    "Ether": {"eps": 4.33, "type": "nonpolar"},
    "Hexane": {"eps": 1.89, "type": "nonpolar"}
}

NUCLEOPHILES = ["Ammonia", "Hydroxide", "Cyanide", "Chloride", "Bromide", "Iodide", "Methanol", "Ethanol", "Water"]
TEMPERATURES = ["Low", "Normal", "High"]

FALLBACK_NUCS = {
    "ammonia": "N",
    "hydroxide": "O",
    "cyanide": "C#N",
    "chloride": "Cl",
    "bromide": "Br",
    "iodide": "I",
    "water": "O",
    "methanol": "CO",
    "ethanol": "CCO"
}

# ------------------ OPSIN ------------------
def iupac_to_smiles(name):
    if not name: return None
    try:
        url = f"https://opsin.ch.cam.ac.uk/opsin/{requests.utils.quote(name)}.json"
        r = requests.get(url, timeout=8)
        if r.status_code == 200:
            return r.json().get("smiles")
    except Exception:
        pass
    return None

def get_smiles_from_input(name, fallback_dict=None):
    """Convert IUPAC to SMILES, fallback to dictionary"""
    smiles = iupac_to_smiles(name)
    if not smiles and fallback_dict:
        smiles = fallback_dict.get(name.lower())
    return smiles

# ------------------ Helpers ------------------
def best_leaving_group(mol):
    priority = {"I": 4, "Br": 3, "Cl": 2, "F": 1}
    lgs = [a for a in mol.GetAtoms() if a.GetSymbol() in priority]
    if lgs: return max(lgs, key=lambda a: priority.get(a.GetSymbol(),0))
    # alcohols
    for atom in mol.GetAtoms():
        if atom.GetSymbol() == "O":
            for nbr in atom.GetNeighbors():
                if nbr.GetSymbol() == "C" and atom.GetHybridization()==Chem.HybridizationType.SP3:
                    if not any(b.GetBondType()==Chem.BondType.DOUBLE and b.GetOtherAtom(nbr).GetSymbol()=="O" for b in nbr.GetBonds()):
                        return atom
    return None

def stability_score(atom):
    deg = sum(1 for n in atom.GetNeighbors() if n.GetSymbol()=="C")
    score = [0,1,2,3][deg]
    for n in atom.GetNeighbors():
        if n.GetIsAromatic(): score +=2
        if any(b.GetBondType()==Chem.BondType.DOUBLE for b in n.GetBonds()): score +=1
    return score

def rearrangements(mol, idx):
    atom = mol.GetAtomWithIdx(idx)
    curr_score = stability_score(atom)
    sites = {idx:("no shift", curr_score)}
    for nbr in atom.GetNeighbors():
        if nbr.GetSymbol()!="C": continue
        nbr_score = stability_score(nbr)
        if nbr_score>curr_score:
            shift = "hydride shift" if nbr.GetDegree()<atom.GetDegree() else "alkyl shift"
            sites[nbr.GetIdx()] = (shift,nbr_score)
    for nbr in atom.GetNeighbors():
        if nbr.GetIsAromatic() or any(b.GetBondType()==Chem.BondType.DOUBLE for b in nbr.GetBonds()):
            nbr_score = stability_score(nbr)
            if nbr_score>curr_score:
                sites[nbr.GetIdx()] = ("resonance",nbr_score)
    return sites

def products(substrate_smiles, nuc_smiles, rearr=False):
    mol = Chem.MolFromSmiles(substrate_smiles)
    if mol is None: return []
    lg = best_leaving_group(mol)
    if lg is None: return []
    c = next((n for n in lg.GetNeighbors() if n.GetSymbol()=="C"), None)
    if c is None: return []
    ci, li = c.GetIdx(), lg.GetIdx()

    em = Chem.EditableMol(mol)
    em.RemoveAtom(li)
    core = em.GetMol()
    if li<ci: ci-=1

    nuc = Chem.MolFromSmiles(nuc_smiles)
    if nuc is None: return []
    attach_candidates = [a.GetIdx() for a in nuc.GetAtoms() if a.GetSymbol() in ("N","O","S")] or [0]
    targets = {ci:("no shift",stability_score(c))}
    if rearr: targets = rearrangements(core,ci)

    out=[]
    for target_idx,(shift,score) in targets.items():
        combo = Chem.CombineMols(core,nuc)
        em2 = Chem.EditableMol(combo)
        em2.AddBond(target_idx, core.GetNumAtoms()+attach_candidates[0], Chem.BondType.SINGLE)
        prod = em2.GetMol()
        try: Chem.SanitizeMol(prod)
        except: pass
        out.append((prod,shift,score))
    return out or []

def elimination_products(substrate_smiles):
    mol = Chem.MolFromSmiles(substrate_smiles)
    if mol is None: return []
    lg = best_leaving_group(mol)
    if lg is None: return []
    c = next((n for n in lg.GetNeighbors() if n.GetSymbol()=="C"), None)
    if c is None: return []
    ci, li = c.GetIdx(), lg.GetIdx()
    # simple beta elimination (choose first beta H)
    beta_h = None
    for nbr in c.GetNeighbors():
        if nbr.GetSymbol()=="C":
            for h in nbr.GetNeighbors():
                if h.GetSymbol()=="H":
                    beta_h = nbr.GetIdx()
                    break
        if beta_h: break
    if not beta_h: return []
    em = Chem.EditableMol(mol)
    em.RemoveAtom(li)
    core = em.GetMol()
    if li<ci: ci-=1
    em2 = Chem.EditableMol(core)
    try:
        em2.AddBond(ci,beta_h,Chem.BondType.DOUBLE)
        prod = em2.GetMol()
        Chem.SanitizeMol(prod)
        return [prod]
    except: return []

# ------------------ Prediction ------------------
def predict(smiles, solvent_name, nuc_strength, temp="Normal"):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None, None
    lg_atom = best_leaving_group(mol)
    if lg_atom is None: return None, mol

    sol = SOLVENTS.get(solvent_name.title())
    if not sol: return None, mol
    eps, stype = sol["eps"], sol["type"]
    c = next((n for n in lg_atom.GetNeighbors() if n.GetSymbol()=="C"), None)
    if c is None: return None, mol
    deg = sum(1 for n in c.GetNeighbors() if n.GetSymbol()=="C")

    sn1 = 3 if deg==3 else 1 if deg==2 else 0
    sn2 = 3 if deg==1 else 1 if deg==2 else 0
    benzylic = any(n.GetIsAromatic() for n in c.GetNeighbors())
    allylic = any(b.GetBondType()==Chem.BondType.DOUBLE for b in c.GetBonds())

    if benzylic or allylic:
        sn1 +=3
        sn2 +=1

    if eps>=50: sn1+=3
    elif eps>=30: sn1+=2
    elif eps>=15: sn1+=1
    elif eps<10: sn2+=1

    if stype=="polar_protic": sn1+=1
    if stype=="polar_aprotic": sn2+=1

    if nuc_strength=="strong": sn2+=2
    elif nuc_strength=="weak": sn1+=1

    tot = sn1+sn2 or 1

    # Adjust for high temperature favoring elimination
    favor_elim = (temp=="High")

    return {"SN1":round(100*sn1/tot,1), "SN2":round(100*sn2/tot,1), "E1/E2": 50 if favor_elim else 0}, mol

# ------------------ Export ------------------
def save_csv(results, filename):
    with open(filename,"w",newline="") as f:
        w = csv.writer(f); w.writerow(["pathway","product_smiles","probability"])
        for path,smi,prob in results: w.writerow([path,smi,prob])

def save_image(rdkit_mols, legends, filename):
    img = Draw.MolsToImage(rdkit_mols, subImgSize=(250,250), legends=legends)
    img.save(filename)

def mols_to_tkimgs(mols, legends, master=None):
    imgs=[]
    for mol,legend in zip(mols,legends):
        if mol is None: continue
        try:
            d2d = Draw.MolToImage(mol,size=(200,200),legend=legend)
            bio = io.BytesIO(); d2d.save(bio,format="PNG"); bio.seek(0)
            img = Image.open(bio)
            imgs.append(ImageTk.PhotoImage(img,master=master))
        except: continue
    return imgs if imgs else []

# ------------------ GUI ------------------
def run_prediction():
    substrate = entry_substrate.get().strip()
    nucleophile = combo_nuc.get().strip()
    solvent = combo_solvent.get().strip()
    temp = combo_temp.get().strip()
    nuc_strength = combo_strength.get().lower()

    sub_smiles = get_smiles_from_input(substrate)
    if not sub_smiles:
        messagebox.showerror("Error",f"Cannot convert substrate '{substrate}' to SMILES.")
        return
    nuc_smiles = get_smiles_from_input(nucleophile,FALLBACK_NUCS)
    if not nuc_smiles:
        messagebox.showerror("Error",f"Cannot convert nucleophile '{nucleophile}' to SMILES.")
        return

    pred, mol = predict(sub_smiles,solvent,nuc_strength,temp)

    txt_output.delete("1.0",tk.END)
    txt_output.insert(tk.END,f"Mechanism likelihood: {pred}\n")

    final=[]
    sn1_prods = products(sub_smiles,nuc_smiles,rearr=True)
    sn2_prods = products(sub_smiles,nuc_smiles,rearr=False)
    weights=[]
    for p,shift,sc in sn1_prods: weights.append((p,f"SN1 via {shift}",sc*pred["SN1"]))
    for p,_,_ in sn2_prods: weights.append((p,"SN2 (direct)",pred["SN2"]))

    if pred["E1/E2"]>0:
        elim_prods = elimination_products(sub_smiles)
        for p in elim_prods: weights.append((p,"E1/E2",pred["E1/E2"]))

    total = sum(w for _,_,w in weights) or 1
    final = [(Chem.MolToSmiles(p),path,round(100*w/total,1)) for p,path,w in weights]
    final.sort(key=lambda x:-x[2])
    txt_output.insert(tk.END,"\nPredicted products:\n")
    for smi,path,prob in final: txt_output.insert(tk.END,f"{path}: {smi} ({prob}%)\n")

    global last_results,last_mols,last_legends,last_tkimgs
    last_results = final
    last_mols = [mol]+[Chem.MolFromSmiles(s) for s,_,_ in final]
    last_legends = ["Reactant"]+[f"{p} ({pr}%)" for _,p,pr in final]

    for widget in image_frame.winfo_children(): widget.destroy()
    container = ttk.Frame(image_frame); container.grid(row=0,column=0,pady=10)
    container.columnconfigure(tuple(range(len(last_mols))),weight=1)
    last_tkimgs=[]
    for i,(mol_obj,legend) in enumerate(zip(last_mols,last_legends)):
        try:
            img = Draw.MolToImage(mol_obj,size=(220,220),kekulize=True,wedgeBonds=True,legend="")
            img_tk = ImageTk.PhotoImage(img,master=root)
            last_tkimgs.append(img_tk)
            card = ttk.Frame(container,padding=5)
            card.grid(row=0,column=i,padx=8)
            lbl_img = tk.Label(card,image=img_tk,relief="groove",bd=2)
            lbl_img.image=img_tk
            lbl_img.pack()
            tk.Label(card,text=legend,font=("Segoe UI",9,"italic")).pack(pady=3)
        except:
            ph = tk.Label(container,text="X",fg="red",font=("Segoe UI",24,"bold"))
            ph.grid(row=0,column=i,padx=10,pady=10)
    image_frame.update_idletasks()

def export_csv():
    if not last_results: return
    file = filedialog.asksaveasfilename(defaultextension=".csv")
    if file: save_csv(last_results,file); messagebox.showinfo("Saved",f"Results saved to {file}")

def export_img():
    if not last_results: return
    file = filedialog.asksaveasfilename(defaultextension=".png")
    if file: save_image(last_mols,last_legends,file); messagebox.showinfo("Saved",f"Image saved to {file}")

# ------------------ MAIN WINDOW ------------------
root = tk.Tk(); root.title("Reaction Pathway Predictor – SN1/SN2/E1/E2 Analyzer")
frame = ttk.Frame(root); frame.pack(fill="both",expand=True,padx=10,pady=10)

ttk.Label(frame,text="Substrate (IUPAC):").grid(row=0,column=0,sticky="w")
entry_substrate = ttk.Entry(frame,width=40); entry_substrate.grid(row=0,column=1)

ttk.Label(frame,text="Nucleophile:").grid(row=1,column=0,sticky="w")
combo_nuc = ttk.Combobox(frame,values=NUCLEOPHILES,width=37); combo_nuc.grid(row=1,column=1)
combo_nuc.set("Ammonia")

ttk.Label(frame,text="Solvent:").grid(row=2,column=0,sticky="w")
combo_solvent = ttk.Combobox(frame,values=list(SOLVENTS.keys()),width=37); combo_solvent.grid(row=2,column=1)
combo_solvent.set("Water")

ttk.Label(frame,text="Nucleophile strength:").grid(row=3,column=0,sticky="w")
combo_strength = ttk.Combobox(frame,values=["auto","strong","weak"],width=37); combo_strength.grid(row=3,column=1)
combo_strength.set("auto")

ttk.Label(frame,text="Temperature:").grid(row=4,column=0,sticky="w")
combo_temp = ttk.Combobox(frame,values=TEMPERATURES,width=37); combo_temp.grid(row=4,column=1)
combo_temp.set("Normal")

ttk.Button(frame,text="Predict Reaction",command=run_prediction).grid(row=5,column=0,columnspan=2,pady=10)
txt_output = tk.Text(frame,height=15,width=60); txt_output.grid(row=6,column=0,columnspan=2)
image_frame = ttk.Frame(frame); image_frame.grid(row=7,column=0,columnspan=2)

ttk.Button(frame,text="Export CSV",command=export_csv).grid(row=8,column=0,pady=5)
ttk.Button(frame,text="Export Image",command=export_img).grid(row=8,column=1,pady=5)

last_results,last_mols,last_legends,last_tkimgs = None,None,None,None
root.mainloop()
